#### 라이브러리, Key

In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
import json
import pymongo
import requests_oauthlib
import tqdm
import pandas as pd

consumer_key = 'HVivZ0F6bigbiBGa66ZMeYCqB'
consumer_secret = '43iHWnNI2YiGydhqkf4t2wgxgmWfmEtZs7iOIyioTvlVnj7zOm'
access_token_key = '1451900284230197256-EfWERRYD9RDPw2B1iI0U2LGIetIwO4'
access_token_secret = 'NjFy51DMOn0GYYkyz9YyPkMvsjQu9ZQN2DtJYiKPPGQkJ'

#### Twitter Streaming API 실행

In [3]:
twitter = requests_oauthlib.OAuth1Session(consumer_key, consumer_secret, access_token_key, access_token_secret)
uri = "http://stream.twitter.com/1.1/statuses/sample.json"
r = twitter.get(uri, stream = True)
r.raise_for_status()

#### Sampling 된 트윗을 MongoDB에 보관

In [ ]:
mongo = pymongo.MongoClient()
for line in tqdm.tqdm(r.iter_lines(), unit = 'tweets', mininterval = 1):
    if line:
        tweet = json.loads(line)
        tweet['_timestamp'] = datetime.datetime.utcnow().isoformat()
        mongo.twitter.sample.insert_one(tweet)

- 대화형 실행 환경

In [5]:
mongo = pymongo.MongoClient() # MongoDB에 접속

- MongoDB로부터 레코드를 읽어들이는 제너레이터 함수
    - `*args`는 임의의 갯수의 positional arguments를 받음을 의미하며, `**kwargs`는 임의의 갯수의 keyword arguments를 받음을 의미한다. 이 때 `*args`, `**kwargs` 형태로 가변인자를 받는걸 packing이라고 한다.

In [6]:
# def tweets(*args, **kwargs):
#     for tweet in mongo.twitter.sample.find(*args, **kwargs):
#         if 'delete' not in tweet:
#             # 흥미가 있는 필드만 yield한다.
#             yield {
#                 'created_at' : tweet['created_at'],
#                 'text' : tweet['text']
#             }

In [7]:
def tweets_day(*args, **kwargs):
    for tweet in mongo.twitter.sample.find(*args, **kwargs):
        if 'delete' not in tweet:
            # 흥미가 있는 필드만 yield한다.
            yield {
                # 날짜별 추출
                'created_at' : tweet['created_at'][:10],
                'text' : tweet['text']
            }
    print(tweet['created_at'][:10])

In [8]:
pd.DataFrame(tweets_day({'lang':'ko'}))

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 618651390561de72148cd51a, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>

In [ ]:
mongo.twitter.sample.find_one()

In [ ]:
pd.DataFrame(list(mongo.twitter.sample.find(limit=5)))

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession #진입점

In [ ]:
spark = SparkSession.builder \
            .master("local") \
            .appName("Word Count") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()